In [1]:
from keras import models,layers
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt

d:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model=VGG16(weights='imagenet')

In [3]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np
from keras import backend as k

In [4]:
img_path='F:\\study\\ml\\DataSet\\dogs_vs_cats\\creative_commons_elephant.jpg'
img=image.load_img(img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
x.shape

(1, 224, 224, 3)

In [5]:
preds=model.predict(x)
print('predicted:',decode_predictions(preds,top=3)[0])

predicted: [('n02504458', 'African_elephant', 0.83298635), ('n01871265', 'tusker', 0.14927106), ('n02504013', 'Indian_elephant', 0.012977002)]


In [6]:
np.argmax(preds[0])

386

In [7]:
african_elephant_output=model.output[:,386]

In [8]:
last_conv_layer=model.get_layer('block5_conv3')

In [9]:
grads=k.gradients(african_elephant_output,last_conv_layer.output)[0]

In [10]:
pooled_grads=k.mean(grads,axis=[0,1,2])
iterate=k.function([model.input],[pooled_grads,last_conv_layer.output[0]])

pooled_grads_value,conv_layer_value=iterate([x])

In [11]:
for i in range(512):
    conv_layer_value[:,:,i]*=pooled_grads_value[i]
    
heatmap=np.mean(conv_layer_value,axis=-1)

In [12]:
# heatmat=np.maximum(heatmap,0)
# heatmap/=np.max(heatmap)
# plt.matshow(heatmap)

In [13]:
import cv2

In [16]:
img=cv2.imread(img_path)
heatmap=cv2.resize(heatmap,(img.shape[1],img.shape[0]))
heatmap=np.uint8(255 * heatmap)
heatmap=cv2.applyColorMap(heatmap,cv2.COLORMAP_JET)
superimposed_img=heatmap*0.4+img
cv2.imwrite('F:\\study\\ml\\DataSet\\dogs_vs_cats\\elephant_cam.jpg', superimposed_img)

True